In [1]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [85]:
class Module(object):
   
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True

    def forward(self, input):
    
        return self.updateOutput(input)

    def backward(self,input, gradOutput):
       
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput


    def updateOutput(self, input):

        pass

    def updateGradInput(self, input, gradOutput):

        pass

    def accGradParameters(self, input, gradOutput):
      
        pass

    def zeroGradParameters(self):
        
        pass

    def getParameters(self):
        
        return []

    def getGradParameters(self):
    
        return []

    def train(self):

        self.training = True

    def evaluate(self):
        self.training = False

    def __repr__(self):
       
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [96]:
class Sequential(Module):
    def __init__(self):
        super().__init__()
        self.modules = []
        self.n = []  

    def add(self, module):
        self.modules.append(module)

    def updateOutput(self, input):
        self.n = [input] 
        for module in self.modules:
            self.n.append(module.forward(self.n[-1]))  
        self.output = self.n[-1]  
        return self.output

    def backward(self, input, gradOutput):
        k = len(self.modules)
        grad = gradOutput  

        for i in range(k, - 1, -1, -1): 
            grad = self.modules[i].backward(self.n[i], grad)

        self.gradInput = grad
        return self.gradInput

    def zeroGradParameters(self):
        for module in self.modules:
            module.zeroGradParameters()

    def getParameters(self):
        return [module.getParameters() for module in self.modules]

    def getGradParameters(self):
        return [module.getGradParameters() for module in self.modules]

    def __repr__(self):
        return "\n".join(str(module) for module in self.modules)

    def __getitem__(self, idx):
        return self.modules[idx]

    def train(self):
        self.training = True
        for module in self.modules:
            module.train()

    def evaluate(self):
        self.training = False
        for module in self.modules:
            module.evaluate()

# Layers

## 1 (0.2). Linear transform layer
Also known as dense layer, fully-connected layer, FC-layer, InnerProductLayer (in caffe), affine transform
- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [87]:
class Linear(Module):
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size=(n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size=n_out)
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)

    def updateOutput(self, input):
        self.output = input @ self.W.T + self.b.T
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput @ self.W
        assert self.gradInput.shape == input.shape
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradW = np.sum(input[:, None, :] * gradOutput[:, :, None], axis=0)
        self.gradb = np.sum(gradOutput, axis=0)

    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)

    def getParameters(self):
        return [self.W, self.b]

    def getGradParameters(self):
        return [self.gradW, self.gradb]

    def __repr__(self):
        s = self.W.shape
        return 'Linear %d -> %d' % (s[1], s[0])

## 2. (0.2) SoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{softmax}(x)_i = \frac{\exp x_i} {\sum_j \exp x_j}$

Recall that $\text{softmax}(x) == \text{softmax}(x - \text{const})$. It makes possible to avoid computing exp() from large argument.

In [88]:
class SoftMax(Module):
    def __init__(self):
        super().__init__()

    def updateOutput(self, input):
        shifted_input = input - np.max(input, axis=1, keepdims=True)
        exp_vals = np.exp(shifted_input)
        self.output = exp_vals / np.sum(exp_vals, axis=1, keepdims=True)
        return self.output

    def updateGradInput(self, input, gradOutput):
        softmax_grad = self.output * gradOutput
        self.gradInput = softmax_grad - self.output * np.sum(softmax_grad, axis=1, keepdims=True)
        return self.gradInput

    def __repr__(self):
        return "SoftMax"


## 3. (0.2) LogSoftMax
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

$\text{logsoftmax}(x)_i = \log\text{softmax}(x)_i = x_i - \log {\sum_j \exp x_j}$

The main goal of this layer is to be used in computation of log-likelihood loss.

In [89]:
class LogSoftMax(Module):
    def __init__(self):
        super().__init__()

    def updateOutput(self, input):
        stable_input = input - np.max(input, axis=1 , keepdims=True)
        log_sum_exp = np.log(np.sum(np.exp(stable_input), axis=1, keepdims=True))
        self.output = stable_input - log_sum_exp
        return self.output

    def updateGradInput(self, input, gradOutput):
        softmax_vals = np.exp(self.output)
        self.gradInput = gradOutput - softmax_vals * np.sum(gradOutput, axis=1, keepdims=True)
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"


In [90]:
class LogSoftMax(Module):
    def __init__(self):
         super(LogSoftMax, self).__init__()

    def updateOutput(self, input):
        self.output = np.subtract(input, input.max(axis=1, keepdims=True))
        self.output = self.output - np.log(np.sum(np.exp(self.output), axis=1, keepdims=True))
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput - np.exp(self.output) * np.sum(gradOutput,  axis=1, keepdims=True)
        return self.gradInput

    def __repr__(self):
        return "LogSoftMax"

## 4. (0.3) Batch normalization
One of the most significant recent ideas that impacted NNs a lot is [**Batch normalization**](http://arxiv.org/abs/1502.03167). The idea is simple, yet effective: the features should be whitened ($mean = 0$, $std = 1$) all the way through NN. This improves the convergence for deep models letting it train them for days but not weeks. **You are** to implement the first part of the layer: features normalization. The second part (`ChannelwiseScaling` layer) is implemented below.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

The layer should work as follows. While training (`self.training == True`) it transforms input as $$y = \frac{x - \mu}  {\sqrt{\sigma + \epsilon}}$$
where $\mu$ and $\sigma$ - mean and variance of feature values in **batch** and $\epsilon$ is just a small number for numericall stability. Also during training, layer should maintain exponential moving average values for mean and variance:
```
    self.moving_mean = self.moving_mean * alpha + batch_mean * (1 - alpha)
    self.moving_variance = self.moving_variance * alpha + batch_variance * (1 - alpha)
```
During testing (`self.training == False`) the layer normalizes input using moving_mean and moving_variance.

Note that decomposition of batch normalization on normalization itself and channelwise scaling here is just a common **implementation** choice. In general "batch normalization" always assumes normalization + scaling.

In [91]:
class BatchNormalization(Module):
    EPS = 1e-3
    def __init__(self, alpha = 0.):
        super(BatchNormalization, self).__init__()
        self.alpha = alpha
        self.moving_mean = None
        self.moving_variance = None

    def updateOutput(self, input):
        # Your code goes here. ################################################
        # use self.EPS please
        if self.training:
            batch_mean = input.mean(axis=0)
            batch_var = input.var(axis=0)

            self.output = (input - batch_mean) / np.sqrt(batch_var + self.EPS)

            if self.moving_mean is None:
                self.moving_mean = batch_mean
                self.moving_variance = batch_var
            else:
                self.moving_mean = self.alpha * self.moving_mean + (1 - self.alpha) * batch_mean
                self.moving_variance = self.alpha * self.moving_variance + (1 - self.alpha) * batch_var
        else:
          if self.moving_mean is None:
              self.moving_mean = input.mean(axis=0)
              self.moving_variance = input.var(axis=0)

          self.output = (input - self.moving_mean) / np.sqrt(self.moving_variance + self.EPS)


        return self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        if self.training:
            batch_var = input.var(axis=0) + self.EPS
            std_inv = 1. / np.sqrt(batch_var)

            N = input.shape[0]
            x_centered = input - input.mean(axis=0)

            self.gradInput = (1. / N) * std_inv * (
                N * gradOutput - np.sum(gradOutput, axis=0)
                - x_centered * np.sum(gradOutput * x_centered, axis=0) / batch_var
            )

        else:
            self.gradInput = gradOutput / np.sqrt(self.moving_variance + self.EPS)
        return self.gradInput

    def __repr__(self):
        return "BatchNormalization"

In [92]:
class ChannelwiseScaling(Module):
    """
       Implements linear transform of input y = \gamma * x + \beta
       where \gamma, \beta - learnable vectors of length x.shape[-1]
    """
    def __init__(self, n_out):
        super(ChannelwiseScaling, self).__init__()

        stdv = 1./np.sqrt(n_out)
        self.gamma = np.random.uniform(-stdv, stdv, size=n_out)
        self.beta = np.random.uniform(-stdv, stdv, size=n_out)

        self.gradGamma = np.zeros_like(self.gamma)
        self.gradBeta = np.zeros_like(self.beta)

    def updateOutput(self, input):
        self.output = input * self.gamma + self.beta
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = gradOutput * self.gamma
        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        self.gradBeta = np.sum(gradOutput, axis=0)
        self.gradGamma = np.sum(gradOutput*input, axis=0)

    def zeroGradParameters(self):
        self.gradGamma.fill(0)
        self.gradBeta.fill(0)

    def getParameters(self):
        return [self.gamma, self.beta]

    def getGradParameters(self):
        return [self.gradGamma, self.gradBeta]

    def __repr__(self):
        return "ChannelwiseScaling"

Practical notes. If BatchNormalization is placed after a linear transformation layer (including dense layer, convolutions, channelwise scaling) that implements function like `y = weight * x + bias`, than bias adding become useless and could be omitted since its effect will be discarded while batch mean subtraction. If BatchNormalization (followed by `ChannelwiseScaling`) is placed before a layer that propagates scale (including ReLU, LeakyReLU) followed by any linear transformation layer than parameter `gamma` in `ChannelwiseScaling` could be freezed since it could be absorbed into the linear transformation layer.

## 5. (0.3) Dropout
Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. Here $p$ is probability of an element to be zeroed.

This has proven to be an effective technique for regularization and preventing the co-adaptation of neurons.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch), zero out elements and multiply elements by $1 / (1 - p)$. The latter is needed for keeping mean values of features close to mean values which will be in test mode. When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [93]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p  # Вероятность зануления нейрона
        self.mask = None  # Маска для зануления случайных нейронов

    def updateOutput(self, input):
        if not self.training:
            # Во время тестирования Dropout не применяется, передаём вход как есть
            self.output = input
        else:
            # Генерируем маску: 1 с вероятностью (1 - p), 0 с вероятностью p
            self.mask = np.random.binomial(1, 1 - self.p, size=input.shape)
            
            # Масштабируем выход, чтобы среднее значение оставалось неизменным
            self.output = input * self.mask / (1 - self.p)
        return self.output

    def updateGradInput(self, input, gradOutput):
        # Градиент передается только через "выжившие" нейроны
        self.gradInput = np.multiply(gradOutput, self.mask) / (1 - self.p)
        return self.gradInput

    def __repr__(self):
        return "Dropout"

## 6. (2.0) Conv2d
Implement [**Conv2d**](https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html). Use only this list of parameters: (in_channels, out_channels, kernel_size, stride, padding, bias, padding_mode) and fix dilation=1 and groups=1.

In [95]:

class Conv2d(Module):
    def __init__(self, input_channels, output_channels, kernel_size,
                 stride=1, padding=0, bias=True, padding_mode='zeros'):
        super(Conv2d, self).__init__()

        self.input_channels = input_channels
        self.output_channels = output_channels
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        if padding == 'same':
          self.padding = (int((self.kernel_size[0] - 1) / 2), int((self.kernel_size[1] - 1) / 2)) # сколько надо паддинга с каждой стороны
        else:
          self.padding = (padding, padding) if isinstance(padding, int) else padding
        self.has_bias = bias
        self.padding_mode = padding_mode

        self.weight = np.random.randn(self.output_channels, self.input_channels, self.kernel_size[0], self.kernel_size[1])
        self.gradW = np.zeros_like(self.weight)
        if self.has_bias:
          self.bias = np.random.randn(output_channels)
          self.gradb = np.zeros_like(self.bias)

        if self.padding_mode=='zeros':
          self.mode='constant'
          self.constant_values=0
        elif self.padding_mode=='replicate':
          self.mode='edge'
        elif self.padding_mode=='reflect':
          self.mode='reflect'
    def updateOutput(self, input):
        # Your code goes here. ################################################
        batch_size = input.shape[0]
        input_height, input_width = input.shape[-2:]

        output_height = int(np.floor((input_height + 2 * self.padding[0] - self.kernel_size[0]) / self.stride[0])) + 1  # вычисляет размер выхода после свертки - с учетом паддинга и страйда
        output_width = int(np.floor((input_width + 2 * self.padding[1] - self.kernel_size[1]) / self.stride[1])) + 1  

        self.output = np.zeros((batch_size, self.output_channels, output_height, output_width))

        if self.mode=='constant':
          self.padded_input = np.pad(input, ((0, 0), (0, 0), (self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), mode=self.mode, constant_values=self.constant_values)
        else:
          self.padded_input = np.pad(input, ((0, 0), (0, 0), (self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), mode=self.mode)
        for batch_idx in range(batch_size):
          for channel_out in range(self.output_channels):
            for h_out in range(output_height):
              for w_out in range(output_width):
                # Вычисляем откуда брать "окно" для свертки
                h_start = h_out * self.stride[0] 
                w_start = w_out * self.stride[1]
                # Берем кусок входного тензора 
                receptive_field = self.padded_input[batch_idx, :, h_start:(h_start + self.kernel_size[0]), w_start: (w_start+ self.kernel_size[1])]
                self.output[batch_idx, channel_out, h_out, w_out] = np.sum(receptive_field * self.weight[channel_out])

                if self.has_bias:
                  self.output[batch_idx, channel_out, h_out, w_out] += self.bias[channel_out]
        return self.output


    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        def edge_mode(row, col, input_height, input_width, padding):
          if row < padding[0]:
            original_row = 0 
          elif row >= input_height + padding[0]:
            original_row = input_height - 1
          else:
            original_row = row - padding[0]

          if col < padding[1]:
            original_col = 0
          elif col >= input_width + padding[1]:
            original_col = input_width - 1
          else:
            original_col = col - padding[1]

          return original_row, original_col

        def reflect_mode(row, col, input_height, input_width, padding):
              height = input_height
              width = input_width
              original_row = row
              if row < 0:
                original_row = -row
              elif row >= height:
                original_row = 2 * (height - 1) - row

              original_col = col
              if col < 0:
                original_col = -col
              elif col >= width:
                original_col = 2 * (width - 1) - col
              return original_row, original_col


        input_height, input_width = input.shape[-2:]
        batch_size, output_channels, output_height, output_width = gradOutput.shape

        self.gradInput = np.zeros_like(input)
        for batch_idx in range(batch_size):
          for channel_out in range(self.output_channels):
            for h_out in range(output_height):
              for w_out in range(output_width):
                h_start = h_out * self.stride[0]
                w_start = w_out * self.stride[1]

                for channel_in in range(self.input_channels):
                 for h_kernel in range(self.kernel_size[0]):
                  for w_kernel in range(self.kernel_size[1]):
                    h_in = h_start + h_kernel - self.padding[0]
                    w_in = w_start + w_kernel - self.padding[1]

                    if (0 <= h_in < input_height) and (0 <= w_in < input_width):
                       self.gradInput[batch_idx, channel_in, h_in, w_in] += gradOutput[batch_idx, channel_out, h_out, w_out] * self.weight[channel_out, channel_in, h_kernel, w_kernel]
                    else:
                      if self.mode=='edge':
                          original_h, original_w = edge_mode(h_in+self.padding[0], w_in+self.padding[1], input_height, input_width, self.padding)
                          self.gradInput[batch_idx, channel_in, original_h, original_w] += gradOutput[batch_idx, channel_out, h_out, w_out] * self.weight[channel_out, channel_in, h_kernel, w_kernel]
                      elif self.mode=='reflect':
                          original_h, original_w = reflect_mode(h_in, w_in, input_height, input_width, self.padding)
                          self.gradInput[batch_idx, channel_in, original_h, original_w] += gradOutput[batch_idx, channel_out, h_out, w_out] * self.weight[channel_out, channel_in, h_kernel, w_kernel]

        return self.gradInput

    def accGradParameters(self, input, gradOutput):
        input_height, input_width = input.shape[-2:]
        batch_size, output_channels, output_height, output_width = gradOutput.shape

        self.gradInput = np.zeros_like(input)

        for batch_idx in range(batch_size):
          for channel_out in range(self.output_channels):
            for h_out in range(output_height):
              for w_out in range(output_width):
                h_start = h_out * self.stride[0]
                w_start = w_out * self.stride[1]

                receptive_field = self.padded_input[batch_idx, :, h_start:(h_start + self.kernel_size[0]), w_start: (w_start+ self.kernel_size[1])]

                self.gradW[channel_out] += receptive_field * gradOutput[batch_idx, channel_out, h_out, w_out]
                if self.has_bias:
                  self.gradb[channel_out] += gradOutput[batch_idx, channel_out, h_out, w_out]
        pass

    def zeroGradParameters(self):
        self.gradW.fill(0)
        if self.has_bias:
          self.gradb.fill(0)

    def getParameters(self):
        if self.has_bias:
          return [self.weight, self.bias]
        else: return [self.weight]

    def getGradParameters(self):
        if self.has_bias:
          return [self.gradW, self.gradb]
        else: return [self.gradW]

    def __repr__(self):
        return "Conv2d"

## 7. (0.5) Implement [**MaxPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html) and [**AvgPool2d**](https://pytorch.org/docs/stable/generated/torch.nn.AvgPool2d.html). 
Use only parameters like kernel_size, stride, padding (negative infinity for maxpool and zero for avgpool) and other parameters fixed as in framework.

In [33]:


class MaxPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(MaxPool2d, self).__init__()
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        self.padding = (padding, padding) if isinstance(padding, int) else padding
        self.cache = None
        
    def updateOutput(self, input):
        batch, num_channels, H_in, W_in = input.shape
        H_out = ((H_in + 2*self.padding[0] - self.kernel_size[0]) // self.stride[0] + 1)
        W_out = ((W_in + 2*self.padding[1] - self.kernel_size[1]) // self.stride[1] + 1)
        self.output = np.zeros((batch, num_channels, H_out, W_out))
        self.indices = np.zeros((batch, num_channels, H_out, W_out, 2), dtype=int)
        padding = np.pad(input, ((0,0), (0,0), (self.padding[0], self.padding[0]), (self.padding[1], self.padding[1])), mode='constant')
        
        for b in range(batch):
            for c in range(num_channels):
                for i in range(H_out):
                    for j in range(W_out):
                        h_start = i * self.stride[0]
                        h_end = h_start + self.kernel_size[0]
                        w_start = j * self.stride[1]
                        w_end = w_start + self.kernel_size[1]
                        window = padding[b, c, h_start:h_end, w_start:w_end]
                        self.output[b, c, i, j] = np.max(window)
                        max_idx = np.unravel_index(np.argmax(window), window.shape)
                        self.indices[b, c, i, j] = [h_start + max_idx[0], w_start + max_idx[1]]
        
        self.cache = (batch, num_channels, H_in, W_in)
        return self.output

    def updateGradInput(self, input, gradOutput):
        batch, num_channels, H_in, W_in = self.cache
        out_shape = (batch, num_channels, H_in + 2*self.padding[0], W_in + 2*self.padding[1])
        self.gradInput = np.zeros(out_shape)

        for b in range(batch):
            for c in range(num_channels):
                for i in range(gradOutput.shape[2]):
                    for j in range(gradOutput.shape[3]):
                        h, w = self.indices[b, c, i, j]
                        self.gradInput[b, c, h, w] += gradOutput[b, c, i, j]

        if self.padding[0] > 0 or self.padding[1] > 0:
            self.gradInput = self.gradInput[:, :, self.padding[0]:-self.padding[0], self.padding[1]:-self.padding[1]] if self.padding[0] > 0 and self.padding[1] > 0 else \
                             self.gradInput[:, :, self.padding[0]:-self.padding[0], :] if self.padding[0] > 0 else \
                             self.gradInput[:, :, :, self.padding[1]:-self.padding[1]]
                             
        return self.gradInput

    def __repr__(self):
        return "MaxPool2d"



class AvgPool2d(Module):
    def __init__(self, kernel_size, stride, padding):
        super(AvgPool2d, self).__init__()
        self.kernel_size = (kernel_size, kernel_size) if isinstance(kernel_size, int) else kernel_size
        self.stride = (stride, stride) if isinstance(stride, int) else stride
        self.padding = (padding, padding) if isinstance(padding, int) else padding
        self.cache = None

    def updateOutput(self, input):
        batch, num_channels, H_in, W_in = input.shape
        H_out = ((H_in + 2*self.padding[0] - self.kernel_size[0]) // self.stride[0] + 1)
        W_out = ((W_in + 2*self.padding[1] - self.kernel_size[1]) // self.stride[1] + 1)
        self.output = np.zeros((batch, num_channels, H_out, W_out))
        padding = np.pad(input, ((0,0), (0,0), (self.padding[0],self.padding[0]), (self.padding[1],self.padding[1])), mode='constant')
        
        for b in range(batch):
            for c in range(num_channels):
                for i in range(H_out):
                    for j in range(W_out):
                        h_start = i* self.stride[0]
                        h_end = h_start + self.kernel_size[0]
                        w_start = j * self.stride[1]
                        w_end = w_start + self.kernel_size[1]
                        window = padding[b, c, h_start:h_end, w_start:w_end]
                        self.output[b, c, i, j] = np.mean(window)
        
        self.cache = (batch, num_channels, H_in, W_in)
        return self.output

    def updateGradInput(self, input, gradOutput):
        batch, num_channels, H_in, W_in = self.cache
        out_shape = (batch, num_channels, H_in + 2*self.padding[0], W_in + 2*self.padding[1])
        self.gradInput = np.zeros(out_shape)
        norm_win = 1 / (self.kernel_size[0] * self.kernel_size[1])
        
        for b in range(batch):
            for c in range(num_channels):
                for i in range(gradOutput.shape[2]):
                    for j in range(gradOutput.shape[3]):
                        h_start = i * self.stride[0]
                        h_end = h_start + self.kernel_size[0]
                        w_start = j * self.stride[1]
                        w_end = w_start + self.kernel_size[1]
                        self.gradInput[b, c, h_start:h_end, w_start:w_end] += gradOutput[b, c, i, j] * norm_win
        
        if self.padding[0] > 0 or self.padding[1] > 0:
            self.gradInput = self.gradInput[:, :, self.padding[0]:-self.padding[0], self.padding[1]:-self.padding[1]] if self.padding[0] > 0 and self.padding[1] > 0 else \
                             self.gradInput[:, :, self.padding[0]:-self.padding[0], :] if self.padding[0] > 0 else \
                             self.gradInput[:, :, :, self.padding[1]:-self.padding[1]]
                             
        return self.gradInput

    def __repr__(self):
        return "AvgPool2d"



## 8. (0.3) Implement **GlobalMaxPool2d** and **GlobalAvgPool2d**.
They do not have testing and parameters are up to you but they must aggregate information within channels. Write test functions for these layers on your own.

In [34]:


class GlobalMaxPool2d(Module):
    def __init__(self, keep_dimensions=False):
        super(GlobalMaxPool2d, self).__init__()
        self.keep_dimensions = keep_dimensions
        self.input_cache = None

    def compute_output(self, input_tensor):
        num_batches, num_channels, h, w = input_tensor.shape
        
        self.input_cache = input_tensor
        flattened_input = input_tensor.reshape(num_batches, num_channels, -1)
        self.max_positions = np.argmax(flattened_input, axis=2)
        
        self.output_tensor = np.max(flattened_input, axis=2)
        
        if self.keep_dimensions:
            self.output_tensor = self.output_tensor.reshape(num_batches, num_channels, 1, 1)

        return self.output_tensor

    def compute_gradient(self, input_tensor, output_gradient):
        num_batches, num_channels = output_gradient.shape[:2]
        h, w = input_tensor.shape[-2:]
        
        self.input_gradient = np.zeros_like(input_tensor)
        
        if self.keep_dimensions:
            output_gradient = output_gradient.reshape(num_batches, num_channels)
        
        max_h, max_w = self._compute_max_positions()

        self._propagate_gradients(output_gradient, num_batches, num_channels, max_h, max_w)

        return self.input_gradient

    def _compute_max_positions(self):
        img_width = self.input_cache.shape[3]
        max_h = self.max_positions // img_width
        max_w = self.max_positions % img_width
        return max_h, max_w

    def _propagate_gradients(self, output_gradient, num_batches, num_channels, max_h, max_w):
        batch_idx, channel_idx = np.indices((num_batches, num_channels))
        for b, c in zip(batch_idx.flatten(), channel_idx.flatten()):
            self.input_gradient[b, c, max_h[b, c], max_w[b, c]] = output_gradient[b, c]

    def __repr__(self):
        return f"GlobalMaxPool2d(keepdims={self.keep_dimensions})"


class GlobalAvgPool2d(Module):
    def __init__(self, keep_dimensions=False):
        super(GlobalAvgPool2d, self).__init__()
        self.keep_dimensions = keep_dimensions

    def compute_output(self, input_tensor):
        num_batches, num_channels, h, w = input_tensor.shape
        
        self.output_tensor = np.mean(input_tensor, axis=(2, 3))
        
        if self.keep_dimensions:
            self.output_tensor = self.output_tensor.reshape(num_batches, num_channels, 1, 1)

        return self.output_tensor

    def compute_gradient(self, input_tensor, output_gradient):
        num_batches, num_channels = output_gradient.shape[:2]
        h, w = input_tensor.shape[-2:]
        total_pixels = h * w
        
        self.input_gradient = np.zeros_like(input_tensor)
        
        if self.keep_dimensions:
            output_gradient = output_gradient.reshape(num_batches, num_channels)
        
        for batch in range(num_batches):
            for channel in range(num_channels):
                for y in range(h):
                    for x in range(w):
                        self.input_gradient[batch, channel, y, x] = output_gradient[batch, channel] / total_pixels

        return self.input_gradient

    def __repr__(self):
        return f"GlobalAvgPool2d(keepdims={self.keep_dimensions})"

## 9. (0.2) Implement [**Flatten**](https://pytorch.org/docs/stable/generated/torch.flatten.html)

In [35]:
class Flatten(Module):
    def __init__(self, start_dim=0, end_dim=-1):
        super(Flatten, self).__init__()
        self.start_dim = start_dim
        self.end_dim = end_dim

    def updateOutput(self, input_data):
        self.input_shape = input_data.shape  # Сохраняем исходную форму
        start_dim = self.start_dim
        end_dim = self.end_dim if self.end_dim >= 0 else len(input_data.shape) + self.end_dim
        
        # Вычисляем новую форму
        new_shape = input_data.shape[:start_dim] + (-1,) + input_data.shape[end_dim + 1:]
        self.output = input_data.reshape(new_shape)
        return self.output

    def updateGradInput(self, input_data, grad_output):
        # Восстанавливаем исходную форму градиента
        self.gradInput = grad_output.reshape(self.input_shape)
        return self.gradInput

    def __repr__(self):
        return f"Flatten(start_dim={self.start_dim}, end_dim={self.end_dim})"


# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**):

In [36]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()

    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output

    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput

    def __repr__(self):
        return "ReLU"

## 10. (0.1) Leaky ReLU
Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope.

In [37]:
class LeakyReLU(Module):
    def __init__(self, slope=0.03):
        super(LeakyReLU, self).__init__()
        self.slope_value = slope

    def updateOutput(self, input_data):
        self.output = np.where(input_data > 0, input_data, self.slope_value * input_data)
        return  self.output

    def updateGradInput(self, input_data, grad_output):
        grad_input = np.where(input_data >= 0, 1, self.slope_value)
        self.gradInput = grad_input * grad_output
        return  self.gradInput

    def __repr__(self):
        return "LeakyReLU"

## 11. (0.1) ELU
Implement [**Exponential Linear Units**](http://arxiv.org/abs/1511.07289) activations.

In [38]:
class ELU(Module):
    def __init__(self, alpha = 1.0):
        super(ELU, self).__init__()

        self.alpha = alpha

    def updateOutput(self, input):
        self.output = np.where(input >= 0, input, self.alpha * (np.exp(input) - 1))
        return  self.output

    def updateGradInput(self, input, gradOutput):
        gradInput = np.where(input >= 0, 1, self.alpha * np.exp(input))
        self.gradInput = gradInput * gradOutput
        return self.gradInput

    def __repr__(self):
        return "ELU"

## 12. (0.1) SoftPlus
Implement [**SoftPlus**](https://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29) activations. Look, how they look a lot like ReLU.

In [39]:
class SoftPlus(Module):
    def __init__(self):
        super(SoftPlus, self).__init__()

    def updateOutput(self, input):
 
        self.output = np.log(1 + np.exp(input))
        return  self.output
    def updateGradInput(self, input, gradOutput):
        gradInput = 1 / (1 + np.exp(-input))  # sigmoid(x)
        self.gradInput = gradInput * gradOutput
        return self.gradInput

    def __repr__(self):
        return "SoftPlus"

## 13. (0.2) Gelu
Implement [**Gelu**](https://pytorch.org/docs/stable/generated/torch.nn.GELU.html) activations.

In [40]:
class Gelu(Module):
    def __init__(self):
        super(Gelu, self).__init__()

    def updateOutput(self, input):
        # Your code goes here. ################################################
        x = input
        sqrt_2 = np.sqrt(2)
        self.output = 0.5 * x * (1 + np.math.erf(x / sqrt_2))

        return  self.output

    def updateGradInput(self, input, gradOutput):
        # Your code goes here. ################################################
        x = input
        sqrt_2 = np.sqrt(2)
        sqrt_2pi = np.sqrt(2 * np.pi)

        cdf = 0.5 * (1 + np.math.erf(x / sqrt_2))
        pdf = np.exp(-0.5 * x**2) / sqrt_2pi

        self.gradInput = gradOutput * (cdf + x * pdf)

        return self.gradInput

    def train(self):
        pass

    def __repr__(self):
        return "Gelu"

# Criterions

Criterions are used to score the models answers.

In [41]:


class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None

    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result.

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)

    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want
        to have readable description.
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.
- input:   **`batch_size x n_feats`**
- target: **`batch_size x n_feats`**
- output: **scalar**

In [42]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()

    def updateOutput(self, input, target):
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output

    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

## 14. (0.2) Negative LogLikelihood criterion (numerically unstable)
You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula,
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. Also there is a small hack with adding small number to probabilities to avoid computing log(0).
- input:   **`batch_size x n_feats`** - probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**



In [43]:
class ClassNLLCriterionUnstable(Criterion):
    EPS = 1e-15
    def __init__(self):
        a = super(ClassNLLCriterionUnstable, self)
        super(ClassNLLCriterionUnstable, self).__init__()

    def updateOutput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        self.output = -np.sum(target * np.log(input_clamp)) / input_clamp.shape[0]
        return self.output

    def updateGradInput(self, input, target):

        # Use this trick to avoid numerical errors
        input_clamp = np.clip(input, self.EPS, 1 - self.EPS)
        self.gradInput = -target / input_clamp / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterionUnstable"

## 15. (0.3) Negative LogLikelihood criterion (numerically stable)
- input:   **`batch_size x n_feats`** - log probabilities
- target: **`batch_size x n_feats`** - one-hot representation of ground truth
- output: **scalar**

Task is similar to the previous one, but now the criterion input is the output of log-softmax layer. This decomposition allows us to avoid problems with computation of forward and backward of log().

In [44]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()

    def updateOutput(self, input, target):
       self.output = -np.sum(target * input) / input.shape[0]
       return self.output

    def updateGradInput(self, input, target):
      self.gradInput = -target / input.shape[0]
      return self.gradInput

    def __repr__(self):
        return "ClassNLLCriterion"

1-я часть задания: реализация слоев, лосей и функций активации - 5 баллов. \\
2-я часть задания: реализация моделей на своих классах. Что должно быть:
  1. Выберите оптимизатор и реализуйте его, чтоб он работал с вами классами. - 1 балл.
  2. Модель для задачи мультирегрессии на выбраных вами данных. Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.
  3. Модель для задачи мультиклассификации на MNIST. Использовать свёртки, макспулы, флэттэны, софтмаксы - 1 балла.
  4. Автоэнкодер для выбранных вами данных. Должен быть на свёртках и полносвязных слоях, дропаутах, батчнормах и тд. - 2 балла. \\

Дополнительно в оценке каждой модели будет учитываться:
1. Наличие правильно выбранной метрики и лосс функции.
2. Отрисовка графиков лосей и метрик на трейне-валидации. Проверка качества модели на тесте.
3. Наличие шедулера для lr.
4. Наличие вормапа.
5. Наличие механизма ранней остановки и сохранение лучшей модели.
6. Свитч лося (метрики) и оптимайзера.

# 1.Оптимизатор

In [45]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score  

In [59]:
class Adam:
    def __init__(self, parameters, lr=0.001, beta1=0.9, beta2=0.999, eps=1e-8):
      
        self.parameters = parameters
        self.lr = lr
        self.beta1 = beta1
        self.beta2 = beta2
        self.eps = eps
        self.t = 0
       
        self.m = [np.zeros_like(p) for p in parameters]
        self.v = [np.zeros_like(p) for p in parameters]
        
      
        self.grads = [None for _ in parameters]

    def step(self):
        self.t += 1
        for i, (param, grad) in enumerate(zip(self.parameters, self.grads)):
            if grad is not None:
               
                self.m[i] = self.beta1 * self.m[i] + (1 - self.beta1) * grad
                self.v[i] = self.beta2 * self.v[i] + (1 - self.beta2) * (grad ** 2)
                
                
                m_hat = self.m[i] / (1 - self.beta1 ** self.t)
                v_hat = self.v[i] / (1 - self.beta2 ** self.t)
                
               
                self.parameters[i] -= self.lr * m_hat / (np.sqrt(v_hat) + self.eps)

    def zero_grad(self):
        self.grads = [None for _ in self.parameters]

## Шедулер

In [47]:
class CosineAnnealingLR:
    def __init__(self, optim, T_max, eta_min=0):
        self.optim = optim
        self.T_max = T_max
        self.eta_min = eta_min
        self.epoch = 0
        self.base_lr = optim.lr  

    def step(self):
        self.epoch += 1
        # Косинусная формула
        progress = min(self.epoch / self.T_max, 1.0)
        lr = self.eta_min + 0.5 * (self.base_lr - self.eta_min) * (1 + math.cos(math.pi * progress))
        self.optim.lr = lr

 # 2. Модель для задачи мультирегрессии на выбраных вами данных.
 Использовать FCNN, dropout, batchnorm, MSE. Пробуйте различные фукнции активации. Для первой модели попробуйте большую, среднюю и маленькую модель. - 1 балл.

## 2.1. Данные

In [72]:
X, y = make_regression(n_samples=3000, n_features=10, n_targets=3)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.15, random_state=9)


print(f"Train size: {X_train.shape}, Val size: {X_val.shape}, Test size: {X_test.shape}")


Train size: (2400, 10), Val size: (510, 10), Test size: (450, 10)


In [80]:
class SmallModel(Module):
    def __init__(self):
        super(SmallModel, self).__init__()
        self.model = Sequential()
        self.model.add(Linear(30, 120))
        self.model.add(Dropout(0.5))
        self.model.add(LeakyReLU())
        self.model.add(Linear(120, 3))
        
    def forward(self, input):
        return self.model.forward(input)
        
    def backward(self, input, grad):
        return self.model.backward(input, grad)
        
    def zeroGradParameters(self):
        self.model.zeroGradParameters()

    def getParameters(self):
        return self.model.getParameters()

    def getGradParameters(self):
        return self.model.getGradParameters()

    def train(self):
        self.model.train()

    def eval(self):
        self.model.evaluate()